In [1]:
# 데이터 크롤링과 정제
# 3장 크롤링 시작하기

In [2]:
# 목차
# 단일 도메인 내의 이동
# 전체 사이트 크롤링
# 전체 사이트에서 데이터 수집
# 인터넷 크롤링

In [3]:
# 웹 크롤링 시작하기
# 위키 백과의 여섯 다리
# 케빈 베이컨의 여섯 다리
# 6단계 법칙
# 인간 관계는 6단계만 거치면 지구상 대부분의 사람들과 연결될 수 있다.

In [4]:
# Wikipedia 페이지 가져오기
# 위키백과의 여섯 다리
# Eric Idle부터 Kevin Bacon 닿는 최소 클릭수 찾기
# Eric Idle 위키피디아 URL
# Kevin Bacon 위키피디아 URL


In [12]:
# 임의의 위키 페이지에서 모든 링크 목록 가져오기
from urllib.request import urlopen
from bs4 import BeautifulSoup

html=urlopen('https://en.wikipedia.org/wiki/Kevin_Bacon')
bs=BeautifulSoup(html, 'html.parser')
for link in bs.find_all('a'):
#     print(link.attrs)
    if 'href' in link.attrs:
        print(link.attrs['href'])   # 불필요한 내용이 많음

/wiki/Wikipedia:Protection_policy#semi
#mw-head
#searchInput
/wiki/Kevin_Bacon_(disambiguation)
/wiki/File:Kevin_Bacon_SDCC_2014.jpg
/wiki/Philadelphia,_Pennsylvania
/wiki/Kevin_Bacon_filmography
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
#cite_note-1
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
/wiki/Holly_Near
/wiki/Wikipedia:Citation_needed
http://baconbros.com/
#cite_note-2
#cite_note-actor-3
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Balto_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Animal_House
/wiki/Diner_(1982_film)
/wiki/Tremors_(1990_film)
/wiki/Crazy,_Stupid,_Love
/wiki/Friday_the_13th_(1980_film)
/wiki/Flatliners
/wiki/The_River_Wild
/wiki/Wild_Things_(film)
/wiki/Stir_of_Echoes
/wiki/Hollow_Man
/wiki/Frost/Nixon_(film)
/wiki/X-Men:_First_Class
/wiki/Black_Mass_(film)
/wiki/Patriots_Day_(film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/HBO
/wiki/Taking_Chan

In [14]:
# 단일 도메인 내의 이동 #1
# 필요한 정보만 가져오기 위해, 위키백과 페이지를 분석할 필요
# 위키백과 페이지
# 위키백과 내부 페이지 링크(internal page link)
# 항목 링크(article link)
# 연관 기사 내용 링크
# article link의 공통점
# <div> 태그의 id='bodyContent' 내부에 있음
# URL에는 콜론이 포함되어 있지 않음
# URL은 /wiki/로 시작

In [17]:
# 항목 링크 찾기
# 항목 링크(연관 기사 링크)만 가져오기
# 항목 링크의 3가지 특성을 이용
# 정규식:^(/wiki/)((?!:).)*$
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

html=urlopen('https://en.wikipedia.org/wiki/Kevin_Bacon')
bs=BeautifulSoup(html, 'html.parser')
body_content=bs.find('div',{'id':'bodyContent'})  # bodyContent부분을 검색
#
# ^(정규식 시작)... $(정규식 끝)
# (/wiki/): '/wiki/를 포함
# ((?!:).)*: ':' 콜론이 없는 문자열 및 임의의 문자가 0회 이상 반복되는 문자열 검색

pattern='^(/wiki/)((?!:).)*$'
for link in body_content.find_all('a',href=re.compile(pattern)):
    if 'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Kevin_Bacon_(disambiguation)
/wiki/Philadelphia,_Pennsylvania
/wiki/Kevin_Bacon_filmography
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
/wiki/Holly_Near
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Balto_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Animal_House
/wiki/Diner_(1982_film)
/wiki/Tremors_(1990_film)
/wiki/Crazy,_Stupid,_Love
/wiki/Friday_the_13th_(1980_film)
/wiki/Flatliners
/wiki/The_River_Wild
/wiki/Wild_Things_(film)
/wiki/Stir_of_Echoes
/wiki/Hollow_Man
/wiki/Frost/Nixon_(film)
/wiki/Black_Mass_(film)
/wiki/Patriots_Day_(film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/HBO
/wiki/Taking_Chance
/wiki/Golden_Globe_Award
/wiki/Screen_Actors_Guild_Award
/wiki/Primetime_Emmy_Award
/wiki/Streaming_television
/wiki/I_Love_Dick_(TV_series)
/wiki/Golden_Globe_Award_for_Best_Actor_%E2%80%93_Television_Series_Musical_or

In [18]:
# 링크간 무작위 이동하기: 동작 과정
# getLinks(articleUrl) 함수 작성
# 파라미터: 임의의 /wiki/<article_name> 형태를 받음
# 리턴값: 해당 링크의 모든 URL 목록을 리턴(리스트 형태)

# 동작 과정
# 시작 URL: https://en.wikipedia.org/wiki/Kevin_Bacon
# 시작 URL 내부의 연관 기사 URL을 가져옴
# 연관 기사 URL에서 랜덤하게 하나의 URL 선택
# 선택된 URL로 이동해서 다시 연관 기사 URL을 가져오는 과정 반복
# 무한 반복
# links=getLinks(articleUrl) 호출
# random 수 발생: randint()
# links에서 무작위 링크 선택
# random.randint(a,b)
# 랜덤 숫자 N 리턴
# a <= N <= b

In [19]:
# 링크간 무작위 이동하기: 소스 코드
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())
def getLinks(articleUrl):
    html=urlopen('https://en.wikipedia.org{}'.format(articleUrl))
    bs=BeautifulSoup(html, 'html.parser')
    return bs.find('div',{'id':'bodyContent'}).find_all('a', href=re.compile('^(/wiki/)((?!:).)*$'))

links=getLinks('/wiki/Kevin_Bacon')
while (len(links)) > 0:
    newArticle=links[random.randint(0, len(links)-1)].attrs['href']
    print(newArticle)
    links=getLinks(newArticle)

C:\Users\User\AppData\Local\Temp\ipykernel_13680\2772371669.py:8: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(datetime.datetime.now())


/wiki/Taking_Chance
/wiki/YouTube
/wiki/Openfilm
/wiki/MovieMaker
/wiki/Film
/wiki/Semiotic_democracy
/wiki/Bipartisanship_as_an_ideology
/wiki/Centrism
/wiki/Hanan_Ashrawi
/wiki/C-SPAN
/wiki/High-definition_television
/wiki/PAL-M
/wiki/Digital_multimedia_broadcasting
/wiki/Radio_network
/wiki/Polarization-division_multiplexing
/wiki/Siae_Microelettronica
/wiki/Multiplexing#Telephony
/wiki/Semaphore
/wiki/N%C4%81gar%C4%AB_script
/wiki/Gregg_shorthand
/wiki/Mundari_Bani
/wiki/All_India_Radio
/wiki/Radio_France
/wiki/Disclose
/wiki/MBA_(identifier)


KeyboardInterrupt: 

In [23]:
# 같은 페이지를 두 번 크롤링 하지 않기
# getLinks() 함수 수정: set 사용
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages=set()  # 세트 선언
def getLinks(pageUrl):
    global pages
    html=urlopen('https://en.wikipedia.org{}'.format(pageUrl))
    bs=BeautifulSoup(html, 'html.parser')
    for link in bs.find_all('a', href=re.compile('^(/wiki/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage=link.attrs['href']
                print(newPage)
                pages.add(newPage)
                pages.add(newPage)
                getLinks(newPage)
getLinks('')

/wiki/Wikipedia
/wiki/Wikipedia:Protection_policy#semi
/wiki/Wikipedia:Requests_for_page_protection
/wiki/Wikipedia:Requests_for_page_protection/Administrator_instructions
/wiki/Wikipedia:Protection_policy
/wiki/Wikipedia:Lists_of_protected_pages
/wiki/Wikipedia:Protection_policy#Semi-protection
/wiki/Wikipedia:Perennial_proposals
/wiki/Wikipedia:Reliable_sources/Perennial_sources


KeyboardInterrupt: 

In [ ]:
# 전체 사이트에서 데이터 수집
# 페이지 방문 과정에서 필요한 정보를 추출
# 수집 정보
# 페이지 제복
# 첫 번째 문단
# 편집 페이지 링크 등

# 웹 페이지의 패턴 분석
# 제목: <h1> 태그 사용(하나만 사용)
# body 텍스트: div#bodyContent 태그에 있음
# 첫 번째 문단의 텍스트만 선택
# <div id='mw-content-text'> => <p></p> 태그 사용
# <div> 태그: division(웹 페이지의 내용을 구분하는데 사용)
# <p> 태그: paragraph(하나의 문단을 만들 때 사용)
# 편집 링크는 항목 페이지에만 존재
# li#ca-edit => span => a
# <li>태그: list의 약자 (목록을 만드는 태그)
# <span> 태그: 인라인 컨테이너, 스타일을 나타내기 위해 사용

In [26]:
# 전체 사이트 데이터 수집 소스
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages=set()
def getLinks(pageUrl):
    global pages
    html=urlopen('https://en.wikipedia.org{}'.format(pageUrl))
    bs=BeautifulSoup(html, 'html.parser')
    try:
        print(bs.h1.get_text())
        print(bs.find(id='mw-content-text').find_all('p')[0])
        print(bs.find(id='ca-edit').find('span').find('a').attrs['href'])
    except AttributeError:
        print('this page is missing something! Continuing.')
        
    for link in bs.find_all('a', href=re.compile('^(/wiki/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage=link.attrs['href']
                print('-'*20)
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)
                
getLinks('')

Main Page
<p><b><a href="/wiki/Edward_the_Elder" title="Edward the Elder">Edward the Elder</a></b> (<abbr title="circa">c.</abbr><span style="white-space:nowrap;"> 874</span> –  924) was <a href="/wiki/List_of_English_monarchs" title="List of English monarchs">King of the Anglo-Saxons</a> from 899 until his death. He was the elder son of <a href="/wiki/Alfred_the_Great" title="Alfred the Great">Alfred the Great</a> and his wife <a href="/wiki/Ealhswith" title="Ealhswith">Ealhswith</a>. When Edward succeeded to the throne, the <a href="/wiki/Vikings" title="Vikings">Vikings</a> ruled <a href="/wiki/Northumbria" title="Northumbria">Northumbria</a>, <a href="/wiki/Kingdom_of_East_Anglia" title="Kingdom of East Anglia">East Anglia</a> and eastern <a href="/wiki/Mercia" title="Mercia">Mercia</a>, leaving only <a href="/wiki/Wessex" title="Wessex">Wessex</a> and western Mercia under Anglo-Saxon control. In 910 a Mercian and West Saxon army inflicted a decisive defeat on an invading Northumbr

Wikipedia:Project namespace
<p>Project pages are for information or discussion about Wikipedia. They should be used to allow Wikipedians to better participate in the community, and not used to excess for unrelated purposes nor to bring the project into disrepute.</p>
this page is missing something! Continuing.
--------------------
/wiki/Wikipedia:Policies_and_guidelines
Wikipedia:Policies and guidelines
<p>Wikipedia's <b>policies and guidelines</b> are developed by the community to describe best practices, clarify principles, resolve conflicts, and otherwise further our goal of creating a free, reliable encyclopedia. There is no need to read any policy or guideline pages to start editing. The <a href="/wiki/Wikipedia:Five_pillars" title="Wikipedia:Five pillars">five pillars</a> are a popular summary of the most pertinent principles.
</p>
this page is missing something! Continuing.
--------------------
/wiki/Wikipedia:WikiProject_Politics
Wikipedia:WikiProject Politics
<p>Welcome to <b>

Wikipedia:Graphics Lab/Map workshop
<p>The <b><a href="/wiki/Wikipedia:Graphics_Lab" title="Wikipedia:Graphics Lab">Graphics Lab</a></b> is a project to improve the graphical content of the Wikimedia projects. Requests for image improvements can be added to the workshop pages: <a class="mw-redirect" href="/wiki/Wikipedia:GL/ILL" title="Wikipedia:GL/ILL">Illustrations</a>, <a class="mw-redirect" href="/wiki/Wikipedia:GL/PHOTO" title="Wikipedia:GL/PHOTO">Photographs</a> and <a class="mw-redirect" href="/wiki/Wikipedia:GL/MAP" title="Wikipedia:GL/MAP">Maps</a>. For questions or suggestions one can use the talk pages: <a href="/wiki/Wikipedia_talk:Graphics_Lab" title="Wikipedia talk:Graphics Lab">Talk:Graphics Lab</a>, <a href="/wiki/Wikipedia_talk:Graphics_Lab/Illustration_workshop" title="Wikipedia talk:Graphics Lab/Illustration workshop">Talk:Illustrations</a>, <a href="/wiki/Wikipedia_talk:Graphics_Lab/Photography_workshop" title="Wikipedia talk:Graphics Lab/Photography workshop">Talk:

Wikipedia:Manual of Style
<p class="mw-empty-elt">
</p>
this page is missing something! Continuing.
--------------------
/wiki/Wikipedia:Policies_and_guidelines#guide
Wikipedia:Policies and guidelines
<p>Wikipedia's <b>policies and guidelines</b> are developed by the community to describe best practices, clarify principles, resolve conflicts, and otherwise further our goal of creating a free, reliable encyclopedia. There is no need to read any policy or guideline pages to start editing. The <a href="/wiki/Wikipedia:Five_pillars" title="Wikipedia:Five pillars">five pillars</a> are a popular summary of the most pertinent principles.
</p>
this page is missing something! Continuing.
--------------------
/wiki/Wikipedia:List_of_policies#Procedural
Wikipedia:List of policies
<p class="mw-empty-elt">
</p>
this page is missing something! Continuing.
--------------------
/wiki/Template:Nutshell
Template:Nutshell
<p>This template presents a <a class="mw-redirect" href="/wiki/Wikipedia:TRITE" tit

Wikipedia:WikiProject Council/Guide
<p>A <b>WikiProject</b> is a group of people who want to work as a team to improve Wikipedia. This <a href="/wiki/Wikipedia:Policies_and_guidelines" title="Wikipedia:Policies and guidelines">guideline</a> outlines the accepted practices for starting and maintaining WikiProjects, as well as some tips for how to organize WikiProjects effectively. For more detailed instructions on how to start a WikiProject, see <a href="/wiki/Wikipedia:WikiProject_Council/Guide/WikiProject" title="Wikipedia:WikiProject Council/Guide/WikiProject">Wikipedia:WikiProject Council/Guide/WikiProject</a>.
</p>
this page is missing something! Continuing.
--------------------
/wiki/Wikipedia:WikiProject_Council
Wikipedia:WikiProject Council
<p>The <b>WikiProject Council</b> is a group of Wikipedians that encourage and assist with the development of WikiProjects. The council acts as a central point for inter-WikiProject discussion and collaboration. New to WikiProjects? See <a hr

Wikipedia:Biographies of living persons
<p>Editors must take particular care when adding <b>information about living persons</b> to <em>any</em> <a href="/wiki/Wikipedia" title="Wikipedia">Wikipedia</a> page.<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[a]</a></sup> Such material requires a high degree of sensitivity, and must adhere <em>strictly</em> to all applicable laws in the United States, to this policy, and to Wikipedia's three core content policies:
</p>
this page is missing something! Continuing.
--------------------
/wiki/Wikipedia:Notability_(people)
Wikipedia:Notability (people)
<p class="mw-empty-elt">
</p>
this page is missing something! Continuing.
--------------------
/wiki/Wikipedia:Autobiography
Wikipedia:Autobiography
<p>Writing an <b><a href="/wiki/Autobiography" title="Autobiography">autobiography</a></b> on Wikipedia is an example of <a class="mw-redirect" href="/wiki/Wikipedia:COI" title="Wikipedia:COI">conflict of interest editing</a> and is <i

IndexError: list index out of range